<a href="https://colab.research.google.com/github/sidhu2690/CogniXis/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [2]:
weights= torchvision.models.EfficientNet_B0_Weights.DEFAULT
Model=torchvision.models.efficientnet_b0(weights=weights)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 159MB/s]


In [6]:
pip install torchinfo; pip install cognixis

In [7]:
from torchinfo import summary

summary(
    model=Model,
    input_size=(1, 3, 224, 224),
    col_names=["input_size", "output_size", "num_params", "trainable"]
)

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Trainable
EfficientNet                                            [1, 3, 224, 224]          [1, 1000]                 --                        True
├─Sequential: 1-1                                       [1, 3, 224, 224]          [1, 1280, 7, 7]           --                        True
│    └─Conv2dNormActivation: 2-1                        [1, 3, 224, 224]          [1, 32, 112, 112]         --                        True
│    │    └─Conv2d: 3-1                                 [1, 3, 224, 224]          [1, 32, 112, 112]         864                       True
│    │    └─BatchNorm2d: 3-2                            [1, 32, 112, 112]         [1, 32, 112, 112]         64                        True
│    │    └─SiLU: 3-3                                   [1, 32, 112, 112]         [1, 32, 112, 112]         --                        --
│    └─Sequential: 2-2  

In [8]:
for param in Model.features.parameters():
  param.requires_grad = False

In [9]:
summary(
    model=Model,
    input_size=(1, 3, 224, 224),
    col_names=["input_size", "output_size", "num_params", "trainable"]
)

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Trainable
EfficientNet                                            [1, 3, 224, 224]          [1, 1000]                 --                        Partial
├─Sequential: 1-1                                       [1, 3, 224, 224]          [1, 1280, 7, 7]           --                        False
│    └─Conv2dNormActivation: 2-1                        [1, 3, 224, 224]          [1, 32, 112, 112]         --                        False
│    │    └─Conv2d: 3-1                                 [1, 3, 224, 224]          [1, 32, 112, 112]         (864)                     False
│    │    └─BatchNorm2d: 3-2                            [1, 32, 112, 112]         [1, 32, 112, 112]         (64)                      False
│    │    └─SiLU: 3-3                                   [1, 32, 112, 112]         [1, 32, 112, 112]         --                        --
│    └─Sequential

In [11]:
Model.classifier= nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(1280, 3)
).

In [23]:
Model=Model.to(device)

In [24]:
!wget -q https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip -O pizza_steak_sushi.zip
!unzip -q pizza_steak_sushi.zip -d ./data/

replace ./data/test/steak/296375.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./data/test/steak/354513.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [29]:
train_dir = "/content/data/train"
test_dir = "/content/data/test"

In [30]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.49, 0.45, 0.42], std= [0.22, 0.22, 0.22])
])

In [31]:
train_data=datasets.ImageFolder(root=train_dir, transform= data_transforms)
test_data= datasets.ImageFolder(root=test_dir, transform= data_transforms)

In [32]:
train_loader=DataLoader(train_data, batch_size= 32, shuffle= True)
test_loader= DataLoader(test_data, batch_size= 32, shuffle= True)

In [33]:
device= "cuda" if torch.cuda.is_available() else "cpu"

In [34]:
loss_fn= nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(Model.parameters(),
                            lr=0.001)

In [35]:
from cognixis import train, train_code

In [36]:
train(Model, train_loader, test_loader, loss_fn, optimizer, epochs=5, device= device)

Epoch: 1/5 | Train Loss: 8.4694 | Train Accuracy: 40.44% | Test Loss: 2.7045 | Test Accuracy: 60.00%
Epoch: 2/5 | Train Loss: 7.3489 | Train Accuracy: 69.33% | Test Loss: 2.1583 | Test Accuracy: 81.33%
Epoch: 3/5 | Train Loss: 5.9834 | Train Accuracy: 86.22% | Test Loss: 1.9860 | Test Accuracy: 86.67%
Epoch: 4/5 | Train Loss: 5.7396 | Train Accuracy: 88.00% | Test Loss: 1.8842 | Test Accuracy: 84.00%
Epoch: 5/5 | Train Loss: 5.1806 | Train Accuracy: 89.33% | Test Loss: 1.6895 | Test Accuracy: 88.00%
